# Preparing dataset

The pandas dataframe is translated into a NumPy array, and split into 70% training, 30% testing.
The shapes are

Training
- X: (4922, 18)
- y: (4922,)

Testing
- X_test: (2110, 18)
- y_test: (2110,)

In [110]:
# libraries and reading csv

import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split 
from sklearn import tree # for plotting tree
from sklearn.tree import DecisionTreeClassifier # for building tree
from sklearn.model_selection import cross_val_score

from sklearn import metrics # for displaying roc curve, and calculating TPR, FPR

df = pd.read_csv('churn.csv') # (7032, 19)

All columns and their possible values

| column name | possible values | | column name | possible values | | column name | possible values |
|---|---|-|---|---|-|---|---|
| gender | 'Female', 'Male' | | InternetService | 'DSL', 'Fiber optic', 'No' | | StreamingMovies | 'Yes', 'No', 'No internet service' |
| SeniorCitizen | 0, 1 | | OnlineSecurity | 'Yes', 'No', 'No internet service' | | Contract | 'Month-to-month', 'One year', 'Two year' |
| Partner | 'Yes', 'No' | | OnlineBackup | 'Yes', 'No', 'No internet service' | | PaperlessBilling | 'Yes', 'No' |
| Dependents | 'Yes', 'No' | | DeviceProtection | 'Yes', 'No', 'No internet service' | | PaymentMethod | 'Electronic check' 'Mailed check' 'Bank transfer (automatic)', 'Credit card (automatic)' |
| tenure | integers | | TechSupport | 'Yes', 'No', 'No internet service' | | MonthlyCharges | floating point |
| PhoneService | 'Yes', 'No' | | StreamingTV | 'Yes', 'No', 'No internet service' | | Churn | 'Yes', 'No' |
| MultipleLines | 'No phone service', 'Yes', 'No' |



In [111]:
def pandas_to_numpy(df):
	"""
	Converts the Pandas dataframe to a NumPy array
	"""
	ds = df.values # Pandas to NumPy
	# 0 gender, 2 Partner, 3 Dependents , 5 PhoneService, 6 MultipleLines
	ds[:, 0] = (ds[:, 0] != 'Male') * 1 # Male -> 0, Female -> 1 
	ds[:, [2, 3, 5]] = (ds[:, [2, 3, 5]] == 'Yes') * 1 # No -> 0, Yes -> 1
	ds[:, 6] = (ds[:, 6] == 'Yes') * 1
	ds[:, 6][ds[:, 5] == 0] = -1

	# 7 InternetService, 8 OnlineSecurity, 9 OnlineBackup, 10 DeviceProtection, 11 TechSupport, 12 StreamingTV, 13 StreamingMovies
	ds[:, 7][ds[:, 7] == "No"] = 0
	ds[:, 7][ds[:, 7] == "DSL"] = 1
	ds[:, 7][ds[:, 7] == "Fiber optic"] = 2
	ds[:, 8:14][ds[:, 8:14] == "No"] = 0
	ds[:, 8:14][ds[:, 8:14] == "Yes"] = 1
	ds[:, 8:14][ds[:, 8:14] == "No internet service"] = -1

	# 14 Contract, 15 PaperlessBilling, 16 PaymentMethod, 18 Churn
	ds[:, 14][ds[:, 14] == "Month-to-month"] = 0
	ds[:, 14][ds[:, 14] == "One year"] = 1
	ds[:, 14][ds[:, 14] == "Two year"] = 2
	ds[:, [15, 18]] = (ds[:, [15, 18]] == "Yes") * 1 # No -> 0, Yes -> 1
	ds[:, 16][ds[:, 16] == "Electronic check"] = 0
	ds[:, 16][ds[:, 16] == "Mailed check"] = 1
	ds[:, 16][ds[:, 16] == "Bank transfer (automatic)"] = 2
	ds[:, 16][ds[:, 16] == "Credit card (automatic)"] = 3

	return ds.astype(float)

In [112]:
# confirm that there are no NaN values, and check what values each column can take
if (False):
	print(df.isna().any())
	for c in df.columns:
		print(df[c].unique())

# convert dataframe to NumPy array
ds = pandas_to_numpy(df) # ds = pandas_to_numpy(df)

# check that the conversion didn't go wrong
if (False):
	print(np.max(ds, axis=0))
	print(np.min(ds, axis=0))

# split testing and training (not used in the end because we used CV)
X, X_test, y, y_test = train_test_split(ds[:, 0:-1], ds[:, -1], test_size=0.3, random_state = 2211)

# Decision Tree

2.1 Building the basic tree

In [113]:
# fitting the tree
model_basic = DecisionTreeClassifier(criterion = 'entropy', random_state=2211)
model_basic.fit(X, y)

DecisionTreeClassifier(criterion='entropy', random_state=2211)

In [ ]:
# evaluating the model
score_accuracy = cross_val_score(model_basic, X, y, scoring='accuracy', cv=10)
score_precision = cross_val_score(model_basic, X, y, scoring='precision', cv=10)
score_recall = cross_val_score(model_basic, X, y, scoring='recall', cv=10)
score_auc = cross_val_score(model_basic, X, y, scoring='roc_auc', cv=10)

score_accuracy.mean(), score_precision.mean(), score_recall.mean(), score_auc.mean()

# plot graphs
plt.plot(score_accuracy, label="accuracy")
plt.plot(score_precision, label="precision")
plt.plot(score_recall, label="recall")
plt.plot(score_auc, label="auc")
plt.legend(loc="upper left")

# plot AUC
fpr, tpr, thresholds = metrics.roc_curve(y, model_basic.predict_proba(X)[:, 1])
roc_auc = metrics.auc(fpr, tpr)
display = metrics.RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc, estimator_name='Basic model')
display.plot()
plt.show()

# correctly classifies 4889 out of 4992 samples
if (False):
	print((model.predict_proba(X)[:, 1] == y).sum())

# print the model's depth and number of leaves
model_basic.get_depth(), model_basic.get_n_leaves()

2.2 Adjusting max_depth, min_samples_split, and min_samples_leaf

In [123]:
max_depth = 5
min_samples_split = 100
min_samples_leaf = 100

# build model
model_adjparam = DecisionTreeClassifier(criterion = 'entropy', random_state=2211,
							   max_depth=max_depth,
							   min_samples_split=min_samples_split,
							   min_samples_leaf=min_samples_leaf)
model_adjparam.fit(X, y)

# get accuracy
score_accuracy = cross_val_score(model_adjparam, X, y, scoring='accuracy', cv=10)
score_accuracy.mean()

DecisionTreeClassifier(criterion='entropy', max_depth=5, min_samples_leaf=100,
                       min_samples_split=100, random_state=2211)

In [128]:

model_adjparam.predict_proba(X)[:, 1], model_adjparam.predict(X)

(array([0.13      , 0.29452055, 0.03515625, ..., 0.00316456, 0.09550562,
        0.02912621]),
 array([0., 0., 0., ..., 0., 0., 0.]))